# Import and process brain data from BOLD5000

In [33]:
import numpy as np
import pandas as pd
import os
import glob

# 3 complete subjects in the BOLD5000 dataset, downloaded from here: https://bold5000-dataset.github.io/website/
subjects = ['CSI1','CSI2','CSI3']
regions = ['LHEarlyVis','LHPPA','LHRSC','RHPPA','RHEarlyVis','RHOPA','RHRSC','LHOPA','LHLOC','RHLOC'] 

# NOTE: There are typos in the BOLD5000 dataset for the ROI names. 
# We have edited those names directly on our copy of the BOLD5000 data; 
# https://figshare.com/articles/dataset/BOLD5000_Release_2_0/14456124?file=28404132


# Data folder for .npy files relative to code location on SCANN Lab dropbox
brain_related_data_folder = os.path.join('..','..','..','data','brain_related_v.2.0','BOLD5000_GLMsingle_ROI_betas','py')

# Get all .npy files from that folder
roi_file_list = glob.glob(os.path.join(brain_related_data_folder,'*.npy'))

# Looking for 40 files (10 ROIs per 4 subj)
assert(len(roi_file_list) == 40)


def convert_behavioral_matrices(df):
    
    # Do some processing of the data frame to make it amenable to correlations later.
    imgnames = df['presentedImage'].to_list()
    df = df.iloc[:,1:]
    df = df.transpose()
    df = df.to_numpy()
    
    out_df = pd.DataFrame(df, columns = imgnames)
    
    return out_df, imgnames


# Catscenes data, taken from the linguistic task data:
catscenes_ling = pd.read_csv(os.path.join('..','..','data','lingDirectionsAverage.csv'))
catscenes_ling, catscenes_imgnames = convert_behavioral_matrices(catscenes_ling)

catscenes_analog = pd.read_csv(os.path.join('..','..','data','analogData_37_bins_Average.csv'))
catscenes_analog, analog_imgnames = convert_behavioral_matrices(catscenes_analog)

display(catscenes_analog.head())

,ATM1.jpg,ATM4.jpg,HorseRaceTrack.jpg,RVinside2.jpg,ShowJumping7.jpg,airplanecabin1.jpg,airplanecabin3.jpg,airplanecabin5.jpg,airportTerminal2.jpg,airportTerminal3.jpg,...,windmill2.jpg,wineVineyard2.jpg,wineVineyard8.jpg,wineVineyard9.jpg,winebarrel2.jpg,winebarrel3.jpg,winebarrel4.jpg,wrestlingring7.jpg,yogastudio1.jpg,yogastudio4.jpg
0,-0.459077,-0.79347,-0.594217,-0.275748,-0.678053,-0.217193,-0.505958,-0.483363,-0.582676,-0.713513,...,-0.875465,-0.377609,-0.193481,-0.231386,-0.483767,-0.247637,-0.694133,-0.713275,-0.558093,-0.531031
1,-0.459077,-0.79347,-0.594217,-0.275748,-0.678053,-0.217193,-0.505958,-0.483363,-0.582676,-0.713513,...,-0.875465,-0.377609,-0.193481,-0.231386,-0.483767,-0.247637,-0.694133,-0.713275,-0.558093,-0.531031
2,-0.459077,-0.79347,-0.594217,-0.275748,-0.678053,-0.217193,-0.505958,-0.483363,-0.582676,-0.713513,...,-0.875465,-0.377609,-0.193481,-0.231386,-0.483767,-0.247637,-0.694133,-0.713275,-0.558093,-0.531031
3,-0.459077,-0.79347,-0.594217,-0.275748,-0.678053,-0.217193,-0.505958,-0.483363,-0.582676,-0.713513,...,-0.875465,-0.377609,-0.193481,-0.231386,-0.483767,-0.247637,-0.694133,-0.713275,-0.558093,-0.531031
4,-0.459077,-0.79347,-0.594217,-0.275748,-0.678053,-0.217193,-0.499793,-0.483363,-0.582676,-0.683056,...,-0.767540,-0.377609,-0.193481,-0.231386,-0.483767,-0.247637,-0.694133,-0.690168,-0.558093,-0.527943


## Handle the image names

In [34]:

def process_img_list(bold5000_imgname_file,catscenes_imgnames=catscenes_imgnames,verbose=False):
    """
    A function that will remove duplicates. Previously, BOLD5000 stimlists had "rep"
       appended to each subsequent presentation of an image. Those "rep" prefixes 
       have been removed, so we need to remove duplicates ourselves. 
       We then also need to remove all images that we never presented."""
    
    with open(bold5000_imgname_file,'r') as f:
        img_list = pd.read_csv(f,header=None,names=['imgnames'])
        
        if verbose:
            print(img_list.head())
            print(f'Before dropping duplicates, length = {len(img_list)}')
        
        # Remove all duplicates besides the first image
        img_list.drop_duplicates(keep='first',inplace=True)
        
        if verbose:
            print(f'After dropping duplicates, length = {len(img_list)}')
        
        # Only keep images on the list IF they appeared in the Catscenes study.
        img_list = img_list[img_list['imgnames'].isin(catscenes_imgnames)]
        
        if verbose:
            print(f'After dropping non-catscenes images, length = {len(img_list)}')
        
    return img_list

# List of image names as presented per subject.
bold5000_imgname_folder = os.path.join('..','..','..','data','brain_related_v.2.0','BOLD5000_imgnames')
bold5000_imgname_file_list = glob.glob(os.path.join(bold5000_imgname_folder,'*'))

# Loop through every subjects list of images presented from BOLD5000
img_list_indices = {}
img_list_names = {}
for bold5000_imgname_file in bold5000_imgname_file_list:
    # Initialize a dataframe
    img_list = pd.DataFrame()
    
    # Grab the subject ID from the file name
    subj = bold5000_imgname_file.split(os.sep)[-1].split('_')[0]

    # Turn the raw list from BOLD5000 into the list (in order) of only stimuli we presented in 
    # catscenes, the first time they were presented. 
    img_list = process_img_list(bold5000_imgname_file)
    
    # We save out the indices and the names of the images in separate dictionaries. 
    img_list_names[subj] = img_list['imgnames'].to_list()
    img_list_indices[subj] = img_list.index.to_list()


In [48]:
# Initialize this with a multiindex - https://pandas.pydata.org/docs/user_guide/advanced.html
#brain_corr = pd.DataFrame(index=[subjects,regions,['partial','ling','analog']])

def create_correlation_vector(df):
    
    """
    From the dataframe that is 500 image columns by N rows,
    create a 500x500 correlation matrix. 
    
    From the correlation matrix, return just the lower triangle
    (below the diagonal) as a vector.
    """
    
    # Calculate correlation matrix columns by columns.
    corr_matrix = df.corr()

    # Sort both axes. 
    corr_matrix.sort_index(inplace=True,axis=0)
    corr_matrix.sort_index(inplace=True,axis=1)
    corr_np = corr_matrix.to_numpy()
    
    #https://numpy.org/doc/stable/reference/generated/numpy.tril_indices.html
    # Select just the lower triangle of the matrix, excluding diagonal:    
    lower_triangle_indices = np.tril_indices(corr_matrix.shape[0],-1)
    
    # Convert the lower triangle of the correlation matrix into a 1d vector.
    corr_1d = np.empty(len(lower_triangle_indices))
    corr_1d = np.reshape(corr_np[lower_triangle_indices],corr_np[lower_triangle_indices].size)
    
    return corr_1d, corr_matrix.columns.to_list()

def calculate_brain_behavior_correlation(corr_1d_1,corr_1d_2,order_1,order_2):
    
    assert(order_1==order_2)
    
    return np.corrcoef(corr_1d_1,corr_1d_2)[0,1]


analog_1d, analog_order = create_correlation_vector(catscenes_analog)
ling_1d, ling_order = create_correlation_vector(catscenes_ling)

for file in roi_file_list:
    
    # Get the subj id and the region name from the roi file we loaded in.
    subj = file.split(os.sep)[-1].split('_')[0]
    if subj in subjects:
        region = file.split(os.sep)[-1].split('_')[-1].split('.')[0]

        # Load in the roi file and trim the unused columns. 
        beta = np.load(file)
        beta = beta[img_list_indices[subj],:]
        beta = np.transpose(beta)
        df = pd.DataFrame(beta,columns=img_list_names[subj])

        # Create correlation vector.
        region_1d,region_order = create_correlation_vector(df)

        # Calculate correlation between analog + ROI
        print(f'{calculate_brain_behavior_correlation(region_1d,analog_1d,analog_order,region_order):.4f}, {region}, analog')
        print(f'{calculate_brain_behavior_correlation(region_1d,ling_1d,ling_order,region_order):.4f}, {region}, ling')

0.0025, LHEarlyVis, analog
-0.0112, LHEarlyVis, ling
0.0065, LHLOC, analog
0.0040, LHLOC, ling
-0.0009, LHOPA, analog
0.0031, LHOPA, ling
-0.0110, LHPPA, analog
-0.0026, LHPPA, ling
-0.0047, LHRSC, analog
0.0042, LHRSC, ling
0.0096, RHEarlyVis, analog
-0.0184, RHEarlyVis, ling
0.0049, RHLOC, analog
-0.0027, RHLOC, ling
0.0048, RHOPA, analog
0.0107, RHOPA, ling
-0.0168, RHPPA, analog
-0.0087, RHPPA, ling
-0.0088, RHRSC, analog
0.0134, RHRSC, ling
-0.0008, LHEarlyVis, analog
0.0019, LHEarlyVis, ling
-0.0015, LHLOC, analog
-0.0004, LHLOC, ling
-0.0003, LHOPA, analog
-0.0042, LHOPA, ling
-0.0002, LHPPA, analog
-0.0264, LHPPA, ling
-0.0025, LHRSC, analog
-0.0050, LHRSC, ling
-0.0001, RHEarlyVis, analog
-0.0025, RHEarlyVis, ling
0.0127, RHLOC, analog
0.0147, RHLOC, ling
-0.0020, RHOPA, analog
-0.0109, RHOPA, ling
-0.0067, RHPPA, analog
-0.0263, RHPPA, ling
-0.0023, RHRSC, analog
-0.0164, RHRSC, ling
0.0135, LHEarlyVis, analog
-0.0024, LHEarlyVis, ling
0.0002, LHLOC, analog
0.0117, LHLOC, lin

In [9]:
# TURN THIS INTO A FUNCTION - because we're doing for analog, linguistic AND per region.
lingCorrMatrix.sort_index(inplace=True,axis=0)
lingCorrMatrix.sort_index(inplace=True,axis=1)
ling = lingCorrMatrix.to_numpy()
ling1D = np.reshape(ling,ling.size)




# This is the code to generate our final subject and region level data.
dfPartial['brain'] = regionCorr1D

brainCorr.at[(subject,TR,'analog'),region] = np.corrcoef(analog1D,regionCorr1D)[0,1]

brainCorr.at[(subject,TR,'ling'),region] = np.corrcoef(ling1D,regionCorr1D)[0,1]

import pingouin as pg

brainCorr.at[(subject,TR,'partial'),region] = pg.partial_corr(data=dfPartial, 
                                                             x='analog', 
                                                             y='brain', 
                                                             covar='ling').round(3).r.pearson
    return brainCorr

,fabricstore1.jpg,church6.jpg,elevatorOutside2.jpg,homegarage4.jpg,doctor1.jpg,pasture9.jpg,horsebarn8.jpg,exerciseequipment5.jpg,igloo5.jpg,sidewalk1.jpg,...,hospitalroom3.jpg,minigolf8.jpg,hallway4.jpg,cave4.jpg,frontFoyer.jpg,wineVineyard2.jpg,waitingroom2.jpg,grocerystore2.jpg,boardwalk2.jpg,dinosaur4.jpg
fabricstore1.jpg,1.000000,0.144126,0.046670,0.074104,-0.037320,-0.097124,-0.022789,0.196476,0.061109,0.041431,...,0.069691,0.029018,0.087606,0.158333,-0.012044,-0.159289,-0.006764,0.088616,-0.138163,-0.045166
church6.jpg,0.144126,1.000000,0.116244,0.271614,-0.018719,0.193379,0.225178,0.426336,-0.101551,0.182404,...,0.013445,-0.232829,-0.275143,0.099531,0.130915,-0.066405,0.018623,-0.083731,0.175491,0.214705
elevatorOutside2.jpg,0.046670,0.116244,1.000000,0.192853,0.233806,0.129046,0.025139,0.151784,0.118785,-0.041848,...,-0.030120,0.037072,-0.214824,0.018494,-0.008493,-0.062885,0.070037,0.135040,0.149068,0.017131
homegarage4.jpg,0.074104,0.271614,0.192853,1.000000,0.230818,0.202519,0.052414,0.310539,0.039376,0.007255,...,-0.016209,0.022903,-0.333890,-0.018496,-0.144737,-0.291713,-0.051264,-0.011218,0.129360,0.087653
doctor1.jpg,-0.037320,-0.018719,0.233806,0.230818,1.000000,-0.023840,0.058951,0.093558,0.004223,-0.064162,...,0.024702,-0.043384,-0.056684,-0.057831,0.113135,-0.094775,-0.060862,-0.073344,0.051183,-0.041858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wineVineyard2.jpg,-0.159289,-0.066405,-0.062885,-0.291713,-0.094775,-0.060458,-0.060774,-0.243024,0.013912,0.034222,...,-0.024850,0.004675,0.163505,0.070686,0.060482,1.000000,0.060756,0.046123,-0.099594,-0.039368
waitingroom2.jpg,-0.006764,0.018623,0.070037,-0.051264,-0.060862,0.070435,0.108554,-0.136303,0.002115,0.115945,...,-0.049073,0.010416,-0.108764,0.163600,-0.017453,0.060756,1.000000,0.249120,0.157742,0.003490
grocerystore2.jpg,0.088616,-0.083731,0.135040,-0.011218,-0.073344,0.115932,0.160076,-0.044111,0.014640,0.195911,...,0.040501,-0.038570,-0.083779,0.276696,-0.111188,0.046123,0.249120,1.000000,-0.066481,-0.115584
boardwalk2.jpg,-0.138163,0.175491,0.149068,0.129360,0.051183,0.045236,0.117489,0.066367,-0.163041,0.094749,...,0.023578,-0.152641,-0.131742,-0.146524,0.016088,-0.099594,0.157742,-0.066481,1.000000,0.208054


In [98]:
ling = pd.read_csv('lingDirectionsAverage.csv')

In [99]:
#get 500 images used in behavioral exps
used_img = ling.iloc[:,0] 

In [100]:
for file in file_list:
    beta_inv = np.load(file)
    beta = np.transpose(beta_inv)
    # load below to create original data csv
    #img_file = '{}/{}_imgnames.txt'.format(folder,file[32:36])
    #load below to create 500 image csv
    img_file = '{}/{}_imgnames_REPEAT.txt'.format(folder,file[32:36])
    my_file = open(img_file, "r")

    # reading the file
    data = my_file.read()

    # replacing end splitting the text
    # when newline ('\n') is seen.
    data_into_list = data.split("\n")
    #remove empty last line -> last line is empty is all txt files here
    data_into_list.pop(-1)
    
    df = pd.DataFrame(beta, columns = data_into_list)
    #only load the line below if creating original data csv
    #df.to_csv('npy_csv/{}.csv'.format(file[32:-4]),sep=',')
    
    df_used_img = df[used_img]
    df_used_img.to_csv('npy_csv/{}_500img.csv'.format(file[32:-4]),sep=',')
    
    
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'BOLD5000_GLMsingle_ROI_betas/py/CSI4_imgnames_REPEAT.txt'

### above error is ok since CSI4 does not have all the 500 images used. So 500 image csvs are only created for CSV1,2 and 3
### The 500img files end at column SG which is 501th column (as first column is index)

## Analog load

In [68]:
analog = pd.read_csv('analogData_37_bins_Average.csv')

In [74]:
analog_img_list = analog.iloc[:,0]

In [75]:
analog_d = analog.iloc[:,1:]

In [76]:
analog_d_t = analog_d.transpose()

In [77]:
analog_d_t_np = analog_d_t.to_numpy()

In [78]:
analog_final = pd.DataFrame(analog_d_t_np, columns = analog_img_list)

In [82]:
analog_final.shape

(36, 500)

In [83]:
analog_final.to_csv('analog.csv',sep=',')

In [84]:
analog_final_corr = analog_final.corr()

In [85]:
analog_final_corr

presentedImage,ATM1.jpg,ATM4.jpg,HorseRaceTrack.jpg,RVinside2.jpg,ShowJumping7.jpg,airplanecabin1.jpg,airplanecabin3.jpg,airplanecabin5.jpg,airportTerminal2.jpg,airportTerminal3.jpg,...,windmill2.jpg,wineVineyard2.jpg,wineVineyard8.jpg,wineVineyard9.jpg,winebarrel2.jpg,winebarrel3.jpg,winebarrel4.jpg,wrestlingring7.jpg,yogastudio1.jpg,yogastudio4.jpg
presentedImage,,,,,,,,,,,,,,,,,,,,,
ATM1.jpg,1.000000,0.565262,0.234142,0.199746,0.606473,0.163178,0.260117,0.772762,0.464397,0.320809,...,0.457129,0.691831,0.184805,0.132695,0.773834,0.183432,0.466301,0.175463,0.341356,0.242387
ATM4.jpg,0.565262,1.000000,0.437641,0.186456,0.408795,0.192994,0.222398,0.711498,0.520995,0.512185,...,0.847567,0.472978,0.172831,0.162108,0.620954,0.208457,0.497019,0.665984,0.424796,0.401926
HorseRaceTrack.jpg,0.234142,0.437641,1.000000,0.287553,0.403356,0.271165,0.438576,0.121306,0.385293,0.253454,...,0.121485,0.369103,0.240910,0.255240,0.411251,0.310607,0.582250,0.013057,0.521599,0.360625
RVinside2.jpg,0.199746,0.186456,0.287553,1.000000,0.421274,0.967758,0.546430,0.010584,0.601546,0.166745,...,0.113625,-0.006861,0.973006,0.932706,0.043783,0.983445,0.717856,0.267640,0.915254,0.468631
ShowJumping7.jpg,0.606473,0.408795,0.403356,0.421274,1.000000,0.295621,0.777892,0.263932,0.618862,0.572013,...,0.382402,0.224379,0.344938,0.245907,0.236967,0.346023,0.486102,0.336830,0.579032,0.583490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
winebarrel3.jpg,0.183432,0.208457,0.310607,0.983445,0.346023,0.992704,0.445759,0.038578,0.571394,0.087022,...,0.119049,0.045037,0.968752,0.956173,0.099789,1.000000,0.747091,0.229469,0.895047,0.403675
winebarrel4.jpg,0.466301,0.497019,0.582250,0.717856,0.486102,0.723106,0.345366,0.314365,0.700549,0.194078,...,0.313345,0.573969,0.692372,0.727761,0.570744,0.747091,1.000000,0.202146,0.828000,0.428024
wrestlingring7.jpg,0.175463,0.665984,0.013057,0.267640,0.336830,0.216656,0.337847,0.319189,0.542897,0.689507,...,0.729602,-0.073156,0.227584,0.210313,0.027860,0.229469,0.202146,1.000000,0.394874,0.602414


In [86]:
analog_final_corr.to_csv('analog_corr.csv',sep=',')

## Linguistic data load

In [87]:
ling = pd.read_csv('lingDirectionsAverage.csv')

In [88]:
ling_imgs = ling.iloc[:,0]

In [89]:
ling_d = ling.iloc[:,1:]

ling_d_t = ling_d.transpose()

ling_d_t_np = ling_d_t.to_numpy()
ling_final = pd.DataFrame(ling_d_t_np, columns = ling_imgs)
ling_final.shape

(7, 500)

In [94]:
ling_final.to_csv('ling.csv',sep=',')

In [95]:
ling_final_corr = ling_final.corr()

In [96]:
ling_final_corr

presentedImage,ATM1.jpg,ATM4.jpg,HorseRaceTrack.jpg,RVinside2.jpg,ShowJumping7.jpg,airplanecabin1.jpg,airplanecabin3.jpg,airplanecabin5.jpg,airportTerminal2.jpg,airportTerminal3.jpg,...,windmill2.jpg,wineVineyard2.jpg,wineVineyard8.jpg,wineVineyard9.jpg,winebarrel2.jpg,winebarrel3.jpg,winebarrel4.jpg,wrestlingring7.jpg,yogastudio1.jpg,yogastudio4.jpg
presentedImage,,,,,,,,,,,,,,,,,,,,,
ATM1.jpg,1.000000e+00,-0.492833,0.588235,-0.622745,-0.420084,-4.200840e-01,-3.823529e-01,0.719676,-0.019672,0.139741,...,-0.160422,0.485530,-4.200840e-01,-0.530662,0.575224,-4.200840e-01,-4.609772e-01,7.486371e-01,-0.488094,7.271084e-17
ATM4.jpg,-4.928329e-01,1.000000,-0.385026,0.900896,0.659912,9.165445e-01,1.694113e-01,-0.297511,0.535656,-0.276432,...,0.252008,0.049743,9.165445e-01,0.963293,0.150604,9.165445e-01,6.247580e-01,-5.040161e-01,0.925778,-2.057637e-01
HorseRaceTrack.jpg,5.882353e-01,-0.385026,1.000000,-0.622745,-0.746816,-4.200840e-01,-3.823529e-01,0.454532,-0.570495,-0.512383,...,-0.641689,0.116105,-4.200840e-01,-0.406841,0.441005,-4.200840e-01,-6.507914e-01,2.138963e-01,-0.564019,-6.549210e-01
RVinside2.jpg,-6.227455e-01,0.900896,-0.622745,1.000000,0.661650,9.547859e-01,4.749754e-01,-0.604897,0.670679,-0.005572,...,0.402995,-0.219697,9.547859e-01,0.939498,-0.247717,9.547859e-01,6.617241e-01,-4.221857e-01,0.973124,9.401268e-02
ShowJumping7.jpg,-4.200840e-01,0.659912,-0.746816,0.661650,1.000000,6.111111e-01,9.335201e-02,-0.030056,0.593171,0.271048,...,0.763763,0.309887,6.111111e-01,0.673722,0.015215,6.111111e-01,9.467293e-01,-8.486251e-02,0.740170,4.157397e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
winebarrel3.jpg,-4.200840e-01,0.916544,-0.420084,0.954786,0.611111,1.000000e+00,4.200840e-01,-0.450835,0.702439,-0.073922,...,0.381881,-0.100504,1.000000e+00,0.968475,-0.091287,1.000000e+00,6.454972e-01,-2.545875e-01,0.981156,-1.182242e-16
winebarrel4.jpg,-4.609772e-01,0.624758,-0.650791,0.661724,0.946729,6.454972e-01,2.711631e-01,-0.087304,0.580381,0.186094,...,0.739510,0.253012,6.454972e-01,0.701253,-0.141421,6.454972e-01,1.000000e+00,9.005574e-17,0.750000,3.622844e-01
wrestlingring7.jpg,7.486371e-01,-0.504016,0.213896,-0.422186,-0.084863,-2.545875e-01,-1.870097e-16,0.482063,0.214599,0.508131,...,0.333333,0.345425,-2.545875e-01,-0.385922,0.069722,-2.545875e-01,9.005574e-17,1.000000e+00,-0.276084,4.762897e-01


In [97]:
ling_final_corr.to_csv('ling_corr.csv',sep=',')